In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as T
from IPython.display import clear_output
from PIL import Image
from matplotlib import cm
from time import perf_counter
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from tqdm import tqdm
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torchvision import datasets, transforms

warnings.filterwarnings('ignore')

In [2]:
# def create_simple_conv_cifar() -> nn.Sequential:
#     model = nn.Sequential(
#             nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1),  # 32 x 32 x 32
#             nn.BatchNorm2d(32),
#             nn.ReLU(),
# 
#             nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, padding=1),  # 32 x 32 x 32
#             nn.BatchNorm2d(32),
#             nn.ReLU(),
# 
#             nn.MaxPool2d(2),  # 16 x 16 x 32
#             nn.Dropout2d(p=0.2),
# 
#             nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),  # 16 x 16 x 64
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
# 
#             nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, padding=1),  # 16 x 16 x 64
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
# 
#             nn.MaxPool2d(2),  # 8 x 8 x 32
#             nn.Dropout2d(p=0.2),
# 
#             nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),  # 8 x 8 x 128
#             nn.BatchNorm2d(128),
#             nn.ReLU(),
# 
#             nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1),  # 8 x 8 x 128
#             nn.BatchNorm2d(128),
#             nn.ReLU(),
# 
#             nn.MaxPool2d(2),  # 4 x 4 x 128
#             nn.Dropout2d(p=0.2),
# 
#             nn.Flatten(),
# 
#             nn.Linear(4 * 4 * 128, 512),
#             nn.BatchNorm1d(512),
#             nn.Dropout(p=0.3),
#             nn.ReLU(),
#             nn.Linear(512, 10)
#         )
#     return model

In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torch.optim import Adam
from torch import nn
import torch


def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

def test(model, device, test_loader):
    model.eval()
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    return correct / len(test_loader.dataset)

@torch.inference_mode()
def predict(model: nn.Module, loader: DataLoader, device: torch.device):
    model.eval()
    prediction = torch.empty(0, device=device)
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)

            output = model(x)
            pred = torch.argmax(output, dim=1)
            prediction = torch.cat((prediction, pred))
    return prediction


@torch.inference_mode()
def predict_tta(model: nn.Module, loader: DataLoader, device: torch.device, iterations: int = 2):
    model.eval()
    preds = []
    for i in range(iterations):
        prediction = []
        for x, y in loader:
            x,y = x.to(device), y.to(device)
            output = model(x)
            prediction.append(output)
        preds.append(torch.cat(prediction))
    preds = torch.stack(preds).mean(dim=0)
    preds = torch.argmax(preds, dim=1)  


In [4]:
from torchvision.datasets import CIFAR10
from torchvision import transforms as T

def get_augmentations(train: bool = True) -> T.Compose:
    
    means = (0.49139968, 0.48215841, 0.44653091)
    stds = (0.24703223, 0.24348513, 0.26158784)

    
    if train:
        return T.Compose(
            [
                T.RandomResizedCrop(size=32, scale=(0.8, 1.1)),
                T.RandomHorizontalFlip(p=0.5),
                T.RandomAdjustSharpness(sharpness_factor=2),
                T.ToTensor(),
                T.Normalize(mean=means, std=stds)
            ]
        )
    else:
        return T.Compose(
            [
                T.ToTensor(),
                T.Normalize(mean=means, std=stds)
            ]
        )


In [5]:
import torch.nn as nn
import torch.nn.functional as F

def create_modified3_conv_cifar():
    class Net(nn.Module):
        def __init__(self):
            super(Net, self).__init__()
            self.conv1 = nn.Conv2d(3, 128, 5, padding=2)
            self.conv2 = nn.Conv2d(128, 128, 5, padding=2)
            self.conv3 = nn.Conv2d(128, 256, 3, padding=1)
            self.conv4 = nn.Conv2d(256, 256, 3, padding=1)
            self.pool = nn.MaxPool2d(2, 2)
            self.bn_conv1 = nn.BatchNorm2d(128)
            self.bn_conv2 = nn.BatchNorm2d(128)
            self.bn_conv3 = nn.BatchNorm2d(256)
            self.bn_conv4 = nn.BatchNorm2d(256)
            self.bn_dense1 = nn.BatchNorm1d(1024)
            self.bn_dense2 = nn.BatchNorm1d(512)
            self.dropout_conv = nn.Dropout2d(p=0.25)
            self.dropout = nn.Dropout(p=0.5)
            self.fc1 = nn.Linear(256 * 8 * 8, 1024)
            self.fc2 = nn.Linear(1024, 512)
            self.fc3 = nn.Linear(512, 10)

        def conv_layers(self, x):
            out = F.relu(self.bn_conv1(self.conv1(x)))
            out = F.relu(self.bn_conv2(self.conv2(out)))
            out = self.pool(out)
            out = self.dropout_conv(out)
            out = F.relu(self.bn_conv3(self.conv3(out)))
            out = F.relu(self.bn_conv4(self.conv4(out)))
            out = self.pool(out)
            out = self.dropout_conv(out)
            return out

        def dense_layers(self, x):
            out = F.relu(self.bn_dense1(self.fc1(x)))
            out = self.dropout(out)
            out = F.relu(self.bn_dense2(self.fc2(out)))
            out = self.dropout(out)
            out = self.fc3(out)
            return out

        def forward(self, x):
            out = self.conv_layers(x)
            out = out.view(-1, 256 * 8 * 8)
            out = self.dense_layers(out)
            return out

    return Net()

In [6]:
# def train(model) -> float:
#     model.train()
# 
#     train_loss = 0
#     total = 0
#     correct = 0
# 
#     for x, y in tqdm(train_loader, desc='Train'):
#         x, y = x.to(device), y.to(device)
# 
#         optimizer.zero_grad()
# 
#         output = model(x)
# 
#         loss = loss_fn(output, y)
# 
#         train_loss += loss.item()
# 
#         loss.backward()
# 
#         optimizer.step()
#         
#         _, y_pred = torch.max(output, 1)
#         total += y.size(0)
#         correct += (y_pred == y).sum().item()
# 
#     train_loss /= len(train_loader)
#     accuracy = correct / total
# 
#     return train_loss, accuracy

In [7]:
# @torch.inference_mode()
# def evaluate(model, loader) -> tuple[float, float]:
#     model.eval()
# 
#     total_loss = 0
#     total = 0
#     correct = 0
# 
#     for x, y in tqdm(loader, desc='Evaluation'):
#         x, y = x.to(device), y.to(device)
# 
#         output = model(x)
# 
#         loss = loss_fn(output, y)
# 
#         total_loss += loss.item()
# 
#         _, y_pred = torch.max(output, 1)
#         total += y.size(0)
#         correct += (y_pred == y).sum().item()
# 
#     total_loss /= len(loader)
#     accuracy = correct / total
# 
#     return total_loss, accuracy

In [8]:
# def plot_stats(
#     train_loss: list[float],
#     valid_loss: list[float],
#     train_accuracy: list[float],
#     valid_accuracy: list[float],
#     title: str
# ):
#     plt.figure(figsize=(16, 8))
# 
#     plt.title(title + ' loss')
# 
#     plt.plot(train_loss, label='Train loss')
#     plt.plot(valid_loss, label='Valid loss')
#     plt.legend()
#     plt.grid()
# 
#     plt.show()
# 
#     plt.figure(figsize=(16, 8))
# 
#     plt.title(title + ' accuracy')
#     
#     plt.plot(train_accuracy, label='Train accuracy')
#     plt.plot(valid_accuracy, label='Valid accuracy')
#     plt.legend()
#     plt.grid()
# 
#     plt.show()

In [9]:
# @torch.inference_mode()
# def predict(model: nn.Module, loader: DataLoader, device: torch.device):
#     model.eval()
#     preds = []
#     for x, y in loader:
#         x, y = x.to(device), y.to(device)
#         output = model(x)
#         preds.append(output)
#     return torch.cat(preds)

In [10]:
@torch.inference_mode()
def predict_tta(model: nn.Module, loader: DataLoader, device: torch.device, iterations: int = 2):
    model.eval()
    preds = []
    for i in range(iterations):
        prediction = []
        with torch.no_grad():
            for x,y in loader:
                x, y = x.to(device), y.to(device)
                pred = model(x)
                prediction.append(pred)
            preds.append(torch.cat(prediction))
    preds = torch.stack(preds).mean(dim=0)
    preds = torch.argmax(preds, dim=1)

    return preds


In [11]:
# def whole_train_valid_cycle(model, num_epochs, title):
#     train_loss_history, valid_loss_history = [], []
#     train_accuracy_history, valid_accuracy_history = [], []
# 
#     for epoch in range(num_epochs):
#         train_loss, train_accuracy = train(model)
#         valid_loss, valid_accuracy = evaluate(model, test_loader)
# 
#         train_loss_history.append(train_loss)
#         valid_loss_history.append(valid_loss)
# 
#         train_accuracy_history.append(train_accuracy)
#         valid_accuracy_history.append(valid_accuracy)
# 
#         clear_output()
# 
#         plot_stats(
#             train_loss_history, valid_loss_history,
#             train_accuracy_history, valid_accuracy_history,
#             title
#         )

In [12]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = create_simple_conv_cifar().to(device)
# optimizer = Adam(model.parameters(), lr=1e-3)
# scheduler = StepLR(optimizer, step_size=25)
# loss_fn = nn.CrossEntropyLoss()
# 
# whole_train_valid_cycle(model, 15,'15_ep')
# 


In [13]:
# preds = predict(model=model, loader=test_loader, device=device)
# preds_tta = predict_tta(model=model, loader=test_loader, device=device)
# 
# torch.save(preds, 'ex_10_preds')
# torch.save(preds_tta, 'ex_10_preds_tta')
# torch.save(model.state_dict(), 'ex_10_params')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = create_modified3_conv_cifar().to(device)
optimizer = Adam(model.parameters())

transform = transforms.Compose([transforms.ToTensor()])
train_set = datasets.CIFAR10(root='data', train=True, transform=get_augmentations(train=True))
test_set = datasets.CIFAR10(root='data', train=False, transform=get_augmentations(train=False))
train_loader = DataLoader(train_set, batch_size=64, shuffle=True)
test_loader = DataLoader(test_set, batch_size=1000, shuffle=False)

for epoch in range(1, 300):
    train(model, device, train_loader, optimizer, epoch)
    accuracy = test(model, device, test_loader)
    print(f'Epoch: {epoch}, Accuracy: {accuracy}')
    if accuracy >= 0.902:
        break

predictions = predict_tta(model, test_loader, device)
torch.save(predictions, 'predictions2.pt')

Epoch: 1, Accuracy: 0.627
Epoch: 2, Accuracy: 0.6922
